In [ ]:
!pip install kaggle
!pip install panda

In [7]:
cd stylianosc_hotmail_com/Arxiv

/home/stylianosc_hotmail_com/Arxiv


In [8]:
! mkdir ~/.kaggle
! cp "kaggle.json" ~/.kaggle/
! chmod 600  ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d Cornell-University/arxiv

100%|██████████████████████████████████████▉| 1.01G/1.02G [00:10<00:00, 111MB/s]
100%|███████████████████████████████████████| 1.02G/1.02G [00:10<00:00, 108MB/s]


In [ ]:
!sudo apt-get install unzip

In [ ]:
!unzip arxiv.zip

Archive:  arxiv.zip
  inflating: arxiv-metadata-oai-snapshot.json  

In [ ]:
import json
import pandas as pd

rows = []
for line in open('arxiv-metadata-oai-snapshot.json', 'r'):
    t = json.loads(line)
    rows.append(t)

df = pd.DataFrame(rows)

In [ ]:
df.sort_values("update_date", ascending=False, inplace=True)
df.drop_duplicates(keep="first", subset=['title'], inplace=True)
df.sort_values("id", ascending=True, inplace=True)

In [ ]:
from datetime import datetime

def latest_date_create(list_dicts):
  max_date = datetime.min
  for temp_dict in list_dicts:
    t = datetime.strptime(temp_dict['created'], '%a, %d %b %Y %H:%M:%S %Z')
    if t > max_date: max_date = t
  return max_date.strftime("%d/%m/%Y")
  

In [ ]:
temp = df[["id", "title", "versions", "abstract", "authors"]]
temp['versions'] = temp['versions'].map(latest_date_create)
temp['authors'] = temp['authors'].str.replace(' and ',', ')
temp.rename(columns={"versions": "date"}, inplace=True)

In [ ]:
from mongoDB_API import *

client = MongoDBClient()

# insert_dataset_data
success_num = client.insert_data(temp, "paper", "arxiv", "id")
print("# data inserted ", success_num)

#client.update_data("paper", "arxiv", "my-datase", {"subtitle": "new new subtitle"})

Server not available


ServerSelectionTimeoutError: ignored